In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
V0, L = -15, 5 # Hartree, bohr

In [ ]:
energyGrid = np.linspace(V0 + 1e-10, 0, 1000)
alpha = np.sqrt(-2*energyGrid)
kappa = np.sqrt(2*(energyGrid-V0))
evenFuncTan = alpha - kappa*np.tan(kappa*L)
oddFuncTan  = alpha + kappa/np.tan(kappa*L)

In [ ]:
plt.plot(energyGrid, evenFuncTan, label='Even')
plt.plot(energyGrid, oddFuncTan, label='Odd')
plt.axhline(0,c='grey')
plt.xlim(V0,0)
plt.ylim(-30,30)
plt.legend()
plt.show()

In [ ]:
evenFuncSin = alpha*np.cos(kappa*L) - kappa*np.sin(kappa*L)
oddFuncSin = alpha*np.sin(kappa*L) + kappa*np.cos(kappa*L)

In [ ]:
plt.plot(energyGrid, evenFuncSin, label='Even')
plt.plot(energyGrid, oddFuncSin, label='Odd')
plt.axhline(0,c='grey')
plt.xlim(V0,0)
plt.ylim(-30,30)
plt.legend()
plt.show()

In [ ]:
z = np.sqrt(-2*V0)*L
y = np.linspace(0,z,1000)
evenFuncZ = np.sqrt(z**2 - y**2)*np.cos(y) - y*np.sin(y)
oddFuncZ = np.sqrt(z**2 - y**2)*np.sin(y) + y*np.cos(y)

In [ ]:
plt.plot(y, evenFuncZ, label='Even')
plt.plot(y, oddFuncZ, label='Odd')
plt.axhline(0,c='grey')
plt.xlim(0,z)
plt.ylim(-z,z)
plt.legend()
plt.show()

In [ ]:
semiCircle = np.sqrt(z**2-y**2)
evenFuncYTan = y*np.tan(y)
oddFuncYTan = -y/np.tan(y)

In [ ]:
plt.plot(y, semiCircle)
#plt.plot(y, evenFuncYTan)
plt.plot(y, oddFuncYTan)
plt.ylim(0,1.1*z)
plt.xlim(0,1.1*z)
plt.show()

In [ ]:
totN = int(np.floor(2*z/np.pi) + 1)
yn = np.arange(0,z,np.pi/2)
yn = np.append(yn,z)
evenFunc = lambda y: np.sqrt(z**2-y**2)*np.cos(y) - y*np.sin(y)
oddFunc  = lambda y: np.sqrt(z**2-y**2)*np.sin(y) + y*np.cos(y)

In [ ]:
yGrid = np.linspace(0,z,1000)
plt.plot(yGrid, evenFunc(yGrid), label="Even")
plt.plot(yGrid, oddFunc(yGrid), label="Odd")
plt.plot([0,z],[0,0],'k--')
plt.plot(yn,np.zeros(len(yn)),'o')
#[plt.axvline(x=i, color='grey') for i in yn]
plt.title("Energy zeros")
plt.xlabel("Energy")
plt.ylabel("Function")
plt.ylim(-z,z)
plt.xlim(0,z)
plt.legend()
plt.show()

In [ ]:
def findRootHybridBisectFalsePos(funcToUse, firstEdge, secondEdge, maxNumberIteration, 
                   positionToleranceAbsolute, valueToleranceAbsolute = 0.):
    """ Finds the root of a 1D function with an hybrid method: Bisect + False Positionsecant  """    
    
    fFirstEdge = funcToUse(firstEdge)
    fSecondEdge = funcToUse(secondEdge)
    
    if fFirstEdge*fSecondEdge >= 0: raise Exception("Solution not bracketed")
    
    secant = (firstEdge * fSecondEdge - secondEdge * fFirstEdge)/(fSecondEdge - fFirstEdge)
    fSecant = funcToUse(secant)
    middle = (firstEdge + secondEdge)/2.
    fMiddle = funcToUse(middle)
    error = 10 * positionToleranceAbsolute  
    iteration = 0;
    
    while (error > positionToleranceAbsolute and abs(fSecant) > valueToleranceAbsolute) \
        and iteration < maxNumberIteration:
        iteration += 1
        
        # We want to make sure that the secant is between middle and secondEdge
        if (secant - middle)*(secondEdge - firstEdge) < 0.:
            secondEdge, firstEdge = firstEdge, secondEdge      # Swap the storage of the edges 
            fSecondEdge, fFirstEdge = fFirstEdge, fSecondEdge  # Swap the storage of the values 
        
        # Now we are sure that secant is between middle and (the value stored in) secondEdge
        if fSecondEdge * fSecant <= 0: # then the zero is between secant and secondEdge
            firstEdge = secant
            fFirstEdge = fSecant
        elif fMiddle * fSecant <= 0:   # then the zero is between middle and secant
            firstEdge = middle
            fFirstEdge = fMiddle
            secondEdge = secant
            fSecondEdge = fSecant  
        else:                          # then the zero is between firstEdge and middle
            secondEdge = middle
            fSecondEdge = fMiddle
            
        oldSolution = secant
        secant = (firstEdge * fSecondEdge - secondEdge * fFirstEdge)/(fSecondEdge - fFirstEdge)
        fSecant = funcToUse(secant)
        middle = (firstEdge + secondEdge)/2.
        fMiddle = funcToUse(middle)
        error = abs(secant - oldSolution)
        
    return secant, iteration

In [ ]:
energy = []
totIter = 0
pErr, fErr = 1e-6, 0.0
for i in range(totN):
    if i%2 == 0:
        en = findRootHybridBisectFalsePos(evenFunc, yn[i], yn[i+1], 10, pErr, fErr)
    else:
        en = findRootHybridBisectFalsePos(oddFunc, yn[i], yn[i+1], 10, pErr, fErr)
    energy.append(en[0])
    totIter += en[1]
energy = np.array(energy)
energy = energy**2/L/L/2 + V0
print(totIter)
print(energy)

In [ ]:
squareWell = [[-2*L,-L,-L,L,L,2*L],[0,0,V0,V0,0,0]]
xGrid = np.linspace(-2*L, 2*L, 1000)

def evenPsi(E):
    psi = np.ones(len(xGrid))
    alpha = np.sqrt(-2*E)
    kappa = np.sqrt(2*(E-V0))
    C = np.exp(-alpha*L)/np.cos(kappa*L)
    psi[xGrid < -L] = np.exp(alpha*xGrid[xGrid < -L])
    psi[(xGrid > -L) & (xGrid < L)] = C*np.cos(kappa*xGrid[(xGrid > -L) & (xGrid < L)])
    psi[xGrid > L] = np.exp(-alpha*xGrid[xGrid > L])
    N = np.sqrt(np.trapz(psi**2,xGrid))
    return psi/N

def oddPsi(E):
    psi = np.ones(len(xGrid))
    alpha = np.sqrt(-2*E)
    kappa = np.sqrt(2*(E-V0))
    C = -np.exp(-alpha*L)/np.sin(kappa*L)
    psi[xGrid < -L] = np.exp(alpha*xGrid[xGrid < -L])
    psi[(xGrid > -L) & (xGrid < L)] = C*np.sin(kappa*xGrid[(xGrid > -L) & (xGrid < L)])
    psi[xGrid > L] = -np.exp(-alpha*xGrid[xGrid > L])
    N = np.sqrt(np.trapz(psi**2,xGrid))
    return psi/N

In [ ]:
baseLine = lambda E: np.full(len(xGrid),E)
plt.title("Energy and wavefunctions")
plt.xlabel("x (bohr)")
plt.ylabel("Energy (Ha)")
plt.plot(*squareWell,'k--')
for i in range(totN):
    if i%2 == 0:
        plt.plot(xGrid,evenPsi(energy[i]) + baseLine(energy[i]));
        plt.plot(xGrid, baseLine(energy[i]), '-.',color='grey')
        plt.text(xGrid[-1], energy[i], '{:.3f}'.format(energy[i]))
    else:
        plt.plot(xGrid,oddPsi(energy[i]) + baseLine(energy[i]));
        plt.plot(xGrid, baseLine(energy[i]), '-.',color='grey')
        plt.text(xGrid[-1], energy[i], '{:.3f}'.format(energy[i]))

In [ ]:
def DOS(E, eGrid, sigma = 0.1):
    dos = np.zeros(len(eGrid))
    for e in E:
        dos += np.exp(-(e-eGrid)**2/2/sigma/sigma)
    return dos/sigma/np.sqrt(2*np.pi)

In [ ]:
eGrid = np.linspace(1.2*V0, 1, 1000)
dos = DOS(energy, eGrid, 0.2)
dos1D = 1/np.sqrt(0.5*(eGrid-energy[0]))
plt.plot(eGrid, dos)
plt.plot(eGrid, dos1D)
plt.show()
print(np.trapz(dos,eGrid))